# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.19,87,100,4.69,SH,1712844374
1,1,atafu village,-8.5421,-172.5159,29.13,75,84,2.60,TK,1712844374
2,2,puerto natales,-51.7236,-72.4875,5.25,81,20,0.51,CL,1712844374
3,3,west island,-12.1568,96.8225,27.99,74,40,4.63,CC,1712844375
4,4,sirib,7.1333,125.4072,26.01,87,100,0.92,PH,1712844375


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City")



# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
city_final = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
city_final = city_final.dropna()

# Display sample data
city_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,kailua-kona,19.6406,-155.9956,23.98,87,0,1.54,US,1712844255
87,87,constantia,44.1833,28.6500,24.38,62,0,2.65,RO,1712844247
119,119,sausset-les-pins,43.3314,5.1043,23.56,45,0,4.12,FR,1712844393
167,167,seferihisar,38.1969,26.8390,25.57,47,0,4.25,TR,1712844401
170,170,darwin,-12.4611,130.8418,26.99,66,0,3.60,AU,1712844297
196,196,holualoa,19.6228,-155.9522,21.91,89,0,1.54,US,1712844405
257,257,san luis de la loma,17.2706,-100.8939,26.05,67,0,1.45,MX,1712844416
402,402,acapulco de juarez,16.8634,-99.8901,25.90,78,0,1.54,MX,1712844409
493,493,anahuac,27.2333,-100.1500,21.09,31,0,2.13,MX,1712844321
495,495,semikarakorsk,47.5194,40.8112,25.59,36,0,3.27,RU,1712844326


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,kailua-kona,US,19.6406,-155.9956,87,
87,constantia,RO,44.1833,28.6500,62,
119,sausset-les-pins,FR,43.3314,5.1043,45,
167,seferihisar,TR,38.1969,26.8390,47,
170,darwin,AU,-12.4611,130.8418,66,
196,holualoa,US,19.6228,-155.9522,89,
257,san luis de la loma,MX,17.2706,-100.8939,67,
402,acapulco de juarez,MX,16.8634,-99.8901,78,
493,anahuac,MX,27.2333,-100.1500,31,
495,semikarakorsk,RU,47.5194,40.8112,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey":geoapify_key, 
    "limit": 25
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
sausset-les-pins - nearest hotel: No hotel found
seferihisar - nearest hotel: No hotel found
darwin - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
anahuac - nearest hotel: No hotel found
semikarakorsk - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,kailua-kona,US,19.6406,-155.9956,87,No hotel found
87,constantia,RO,44.1833,28.6500,62,No hotel found
119,sausset-les-pins,FR,43.3314,5.1043,45,No hotel found
167,seferihisar,TR,38.1969,26.8390,47,No hotel found
170,darwin,AU,-12.4611,130.8418,66,No hotel found
196,holualoa,US,19.6228,-155.9522,89,No hotel found
257,san luis de la loma,MX,17.2706,-100.8939,67,No hotel found
402,acapulco de juarez,MX,16.8634,-99.8901,78,No hotel found
493,anahuac,MX,27.2333,-100.1500,31,No hotel found
495,semikarakorsk,RU,47.5194,40.8112,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)